<a href="https://colab.research.google.com/github/BrunoSaintClair/Titanic-Kaggle/blob/main/Submissao2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [352]:
import pandas as pd

In [353]:
df_treino = pd.read_csv('train.csv')
df_teste = pd.read_csv('test.csv')

In [354]:
df_treino.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_teste.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [355]:
print('DF treino:')
print()
display(df_treino.info())
print('-'*30)
print('DF teste:')
print()
display(df_teste.info())

DF treino:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


None

------------------------------
DF teste:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          332 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         417 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


None

# **O que pretendo fazer:**

- Preencher valores vazios da coluna Embarked com o valor que mais se repetiu
  - Após isso, diluir em três colunas númericas com 0 e 1, usando OneHotEncoder

- Age: preencher valores vazios com média da idade por sexo e pclass.

- Sex: passar um labelEncoder


In [356]:
# No DataFrame de teste essa coluna não possui valores vazios.

df_treino['Embarked'].fillna(df_treino['Embarked'].mode().iloc[0], inplace=True)

In [357]:
# Para servir como base para saber se o OneHotEncoder funcionou corretamente:
print('DF treino:')
display(df_treino['Embarked'].value_counts())
print('-'*30)
print('DF teste:')
display(df_teste['Embarked'].value_counts())

DF treino:


Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

------------------------------
DF teste:


Embarked
S    270
C    102
Q     46
Name: count, dtype: int64

In [358]:
#Aplicando o OneHotEncoder para diluir a coluna de Embarked em 3 colunas diferentes possuindo 0 ou 1,
# onde essas 3 colunas representam os 3 valores presentes anteriormente nessa coluna.

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', dtype='int64') # instanciando
ohe = ohe.fit(df_treino[['Embarked']])
ohe.transform(df_treino[['Embarked']]).toarray()
df_ohe = pd.DataFrame(ohe.transform(df_treino[['Embarked']]).toarray(), columns=ohe.get_feature_names_out())

In [359]:
# Verificando se funcionou corretamente:
display(df_ohe.head())
display(df_ohe.value_counts())

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


Embarked_C  Embarked_Q  Embarked_S
0           0           1             646
1           0           0             168
0           1           0              77
Name: count, dtype: int64

In [360]:
# Adicionando as colunas geradas ao nosso DataFrame e eliminando a antiga coluna Embarked
df_treino = pd.concat([df_treino, df_ohe], axis=1)
df_treino.drop('Embarked', axis=1, inplace=True)
df_treino.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,0,0,1


## Repetindo o processo mas com a base de teste:

In [361]:
ohe = OneHotEncoder(handle_unknown='ignore', dtype='int64')
ohe = ohe.fit(df_teste[['Embarked']])

ohe.transform(df_teste[['Embarked']]).toarray()

df_ohe = pd.DataFrame(ohe.transform(df_teste[['Embarked']]).toarray(), columns=ohe.get_feature_names_out())
display(df_ohe.value_counts())



Embarked_C  Embarked_Q  Embarked_S
0           0           1             270
1           0           0             102
0           1           0              46
Name: count, dtype: int64

In [362]:
df_teste = pd.concat([df_teste, df_ohe], axis=1)
df_teste.drop('Embarked', axis=1, inplace=True)

# Preenchendo valores vazios da coluna 'Age':

In [363]:
display(df_treino['Age'].isnull().sum())
display(df_teste['Age'].isnull().sum())

177

86

In [364]:
def preencher_idade(linha):
    sexo = linha['Sex']
    p_classe = linha['Pclass']

    if sexo == 'male':
      return media_idade_homens[p_classe]
    else:
      return media_idade_mulheres[p_classe]

In [365]:
media_idade = df_treino.groupby(['Sex', 'Pclass'])['Age'].mean()

media_idade_mulheres = media_idade['female']
media_idade_homens = media_idade['male']

df_treino['Age'] = df_treino['Age'].fillna(df_treino.apply(preencher_idade, axis=1))

In [366]:
media_idade = df_teste.groupby(['Sex', 'Pclass'])['Age'].mean()

media_idade_mulheres = media_idade['female']
media_idade_homens = media_idade['male']

df_teste['Age'] = df_teste['Age'].fillna(df_teste.apply(preencher_idade, axis=1))

## Utilizando LabelEncoder para transformar valores presentes na coluna 'Sex' em valores númericos.

In [367]:
from sklearn.preprocessing import LabelEncoder

codificador = LabelEncoder()

df_treino['Sex'] = codificador.fit_transform(df_treino['Sex'])
df_teste['Sex'] = codificador.fit_transform(df_teste['Sex'])

display(df_treino.head(3))
display(df_teste.head(3))

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,1,22.0,1,0,7.2500,0,0,1
1,2,1,1,0,38.0,1,0,71.2833,1,0,0
2,3,1,3,0,26.0,0,0,7.9250,0,0,1


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,892,3,1,34.5,0,0,7.8292,0,1,0
1,893,3,0,47.0,1,0,7.0000,0,0,1
2,894,2,1,62.0,0,0,9.6875,0,1,0


## Verificando se ainda existe algum valor vazio ou não númerico:

In [368]:
display(df_treino.info())
display(df_teste.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked_C   891 non-null    int64  
 9   Embarked_Q   891 non-null    int64  
 10  Embarked_S   891 non-null    int64  
dtypes: float64(2), int64(9)
memory usage: 76.7 KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    int64  
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         417 non-null    float64
 7   Embarked_C   418 non-null    int64  
 8   Embarked_Q   418 non-null    int64  
 9   Embarked_S   418 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 32.8 KB


None

In [369]:
df_teste['Fare'] = df_teste['Fare'].fillna(df_teste['Fare'].mean())

# Criando modelo de machine learning

In [370]:
from sklearn.model_selection import train_test_split

X = df_treino.drop(['PassengerId','Survived'],axis=1)
y = df_treino.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

### Árvore de Decisão

In [371]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
previsao_tree = clf.predict(X_test)

### KNN

In [372]:
from sklearn.neighbors import KNeighborsClassifier

clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn = clf_knn.fit(X_train, y_train)
previsao_knn = clf_knn.predict(X_test)

### Regressão logística

In [373]:
from sklearn.linear_model import LogisticRegression

clf_rl = LogisticRegression()
clf_rl = clf_rl.fit(X_train, y_train)
previsao_rl = clf_rl.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Avaliando a acurácia dos modelos:

In [374]:
from sklearn.metrics import accuracy_score

score_tree = accuracy_score(y_test, previsao_tree)
score_knn = accuracy_score(y_test, previsao_knn)
score_rl = accuracy_score(y_test, previsao_rl)

print('Árvore de decisão:', score_tree)
print()
print('KNN:', score_knn)
print()
print('Regressão logística:', score_rl)

Árvore de decisão: 0.726457399103139

KNN: 0.7219730941704036

Regressão logística: 0.8116591928251121


## Gerando arquivo para submeter no Kaggle:

In [375]:
X_test = df_teste.drop('PassengerId', axis=1)

previsao_rl_teste = clf_rl.predict(X_test)

df_teste['Survived'] = previsao_rl_teste

resultado = df_teste[['PassengerId', 'Survived']]

In [376]:
resultado.to_csv('resultado2.csv', index=False)

## Acurácia no Kaggle: 76,08%

### Conclusão:
Preenchi os valores vazios na coluna idade de uma maneira mais específica, já que calculei a média baseando-se no sexo e na classe do passageiro, diferente da primeira submissão onde me baseei apenas pelo sexo, e utilizei a coluna 'Embarked' nesse modelo.

Mas o aumento na acurácia foi de apenas 0,72%.

Na próxima submissão pretendo utilizar a base gerada nesse código aqui, porém alterando alguns parâmetros nos modelos de ML, e testar alguns outros modelos.